In [25]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import norm
import plotly.graph_objects as go

In [26]:
import matplotlib.pyplot as plt

In [27]:
# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [28]:
#pd.reset_option('^display.', silent=True)

In [29]:
# Define the path to your Excel file
file_path = r'C:\Users\skosloff\Documents\Python Scripts\AVGO_Data.xlsx'

# Read the Excel file
data = pd.DataFrame(pd.read_excel(file_path, sheet_name='Sheet1'))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\skosloff\\Documents\\Python Scripts\\AVGO_Data.xlsx'

In [ ]:
data.rename(columns={'Unnamed: 0': 'Date', 'avgo us equity': 'Price'}, inplace=True)
data.set_index('Date', inplace=True)
data = data.iloc[:, :1]

In [30]:
data

NameError: name 'data' is not defined

In [31]:
# Calculate daily returns
data['Daily Return'] = data['Price'].pct_change()

# Calculate historical volatility
data['Volatility'] = data['Daily Return'].std() * np.sqrt(252)

NameError: name 'data' is not defined

In [32]:
# Store variables
sig = data['Volatility']
S = data['Price'] # Spot price
D = 0.0159
E = 1850 # Strike price
T = -(( (data.index - datetime.datetime.today()).days + 1 ) / 365)
r = 0.0531
d1 = ( np.log(S / E) + (r - D + 0.5 * sig**2)*T ) / ( sig * np.sqrt(T) )
d2 = d1 - sig * np.sqrt(T) 

NameError: name 'data' is not defined

In [33]:
# Call option
data['call price'] = S * np.exp(-D * T) * norm.cdf(d1) - ( E * np.exp(-r * T) * norm.cdf(d2) )

# Call delta: sensitivity to underlying
data['c_delta'] = np.exp(-D * T) * norm.cdf(d1) * 100

# Gamma: sensitivity of delta to underlying
data['gamma'] = ( np.exp(-D * T) * norm.pdf(d1) ) / ( sig * S * np.sqrt(T) ) 
#( E * np.exp(-r * T ) ) * ( ( norm.pdf(d2) ) / ( S**2 * sig * np.sqrt(T) ) )

# Call theta: sensititivty to time to expiry
data['c_theta'] = ( ( -sig * S * norm.pdf(d1) / ( 2 * np.sqrt(T) ) ) - ( r * np.exp(-r * T) * E * norm.cdf(d2) ) ) / 252
#data['c_theta'] = ( ( -sig * S * np.exp(-d1**2 / 2 ) / ( 2 * np.sqrt(T) * np.sqrt(2 * np.pi) ) ) - ( r * np.exp(-r * T) * E * norm.cdf (d2) ) ) /252 # Asim

# Vega: sensitivity to volatility
data['vega'] = S * np.sqrt(T) * norm.pdf(d1) / 100

# Call rho (IR): sensitivity to interest rate
data['c_rho'] = ( E * T * np.exp(-r * T) * norm.cdf(d2) ) / 100

# Put option
data['put price'] = -S * np.exp(-D * T) * norm.cdf(-d1) + E * np.exp(-r * T) * norm.cdf(-d2)

# Put delta
data['p_delta'] = np.exp(-D * T) * (norm.cdf(d1) - 1) * 100

# Put theta
data['p_theta'] = ( ( -sig * S * norm.pdf(d1) / ( 2 * np.sqrt(T) ) ) + ( r * np.exp(-r * T) * E * norm.cdf(-d2) ) ) / 252
#( ( -sig * S * np.exp(-D * T) * norm.pdf(d1) ) / ( 2 * np.sqrt(T) ) ) - ( D * S * norm.cdf(d1) * np.exp(-D * T) ) + ( r * E * np.exp(-r * T) * norm.cdf(d2) ) # Willmott

# Put rho (IR)
data['p_rho'] = ( -E * T * np.exp(-r * T) * norm.cdf(-d2) ) / 100

NameError: name 'S' is not defined

In [34]:
data

NameError: name 'data' is not defined

In [35]:
plt.plot(data['Price'], data['call price'])

NameError: name 'data' is not defined

In [36]:
T

NameError: name 'T' is not defined

### Strategy Table

#### OTM Put Table

In [37]:
# Calculates payoff at expiry of %OTM put given some % move in the underlying
# Note that moves in the underlying and %OTM are input as positive numbers
def OTMput_T(S_0, move_und, OTM, prem):
    E = S_0 * (1 - OTM / 100) # Strike as %OTM of S_0
    S_T = S_0 * (1 - move_und / 100) # Underlying price at expiry as % move in spot price
    #print("Strike:", E)
    #print("S_T:", S_T)
    return max(0, (E - S_T) / prem)

In [38]:
OTMput_T(554.99, 5, 3, 2.09)

5.310909090909072

In [39]:
# Give spot price
S_0 = 554.99

# Define table column and row headers
percent_moves = np.array([2.5, 5, 7.5, 10])
percent_OTMs = np.array([0, 3, 5, 10, 15])

# Array of premia in dollars
prems = np.array([6.33, 2.09, 1.09, 0.37, 0.08])


# Create empty dataframe with moves in underlying as columns
OTMput_table = pd.DataFrame(index=percent_OTMs, columns=percent_moves)

# Update the dataframe with the new titles for columns and rows
OTMput_table.columns.name = 'Moves in Underlying'
OTMput_table.index.name = '% OTM'

for i in OTMput_table.index:
    for j in OTMput_table.columns:
        OTMput_table.at[i, j] = OTMput_T(S_0, j, i, prems[np.where(OTMput_table.index == i)[0][0]])

In [40]:
OTMput_table

Moves in Underlying,2.5,5.0,7.5,10.0
% OTM,,,,
0,2.191904,4.383807,6.575711,8.767615
3,0,5.310909,11.949545,18.588182
5,0,0,12.729128,25.458257
10,0,0,0,0
15,0,0,0,0


In [41]:
prems[np.where(OTMput_table.index == 15)[0][0]]

0.08

#### Put Spread Table

In [42]:
# Calculates payoff at expiry of a %X-%Y put spread given some % move in the underlying
def SpreadPut_T(S_0, move_und, OTM_long, OTM_short, prem_long, prem_short):
    # Calculate strike prices for the long and short puts
    E_long = S_0 * (1 - OTM_long / 100)  # Strike price for the long put
    E_short = S_0 * (1 - OTM_short / 100)  # Strike price for the short put
    
    # Calculate the underlying price at expiry given the move in the underlying
    S_T = S_0 * (1 - move_und / 100)
    
    # Calculate the payoff of the long put
    long_put_payoff = max(0, (E_long - S_T) / prem_long)
    
    # Calculate the payoff of the short put
    short_put_payoff = max(0, (E_short - S_T) / prem_short)
    
    # Net payoff of the put spread
    SpreadPut_payoff = long_put_payoff - short_put_payoff
    
    return SpreadPut_payoff

In [43]:
SpreadPut_T(S_0, 5, 97, 90, 1, 1)

0

#### OTM Call Table

In [44]:
# Calculates payoff at expiry of %OTM call given some % move in the underlying
# Note that moves in the underlying and %OTM are input as positive numbers
def OTMcall_T(S_0, move_und, OTM, prem):
    E = S_0 * (1 + OTM / 100) # Strike as %OTM of S_0
    S_T = S_0 * (1 + move_und / 100) # Underlying price at expiry as % move in spot price
    #print("Strike:", E)
    #print("S_T:", S_T)
    return max(0, (S_T - E) / prem)

In [45]:
# Give spot price
S_0 = 554.99

# Define table column and row headers
percent_moves = np.array([2.5, 5, 7.5, 10])
percent_OTMs = np.array([3, 5, 10, 15])

# Array of premia in dollars
prems = np.array([1.64, 0.28, 0.08, 0.04])


# Create empty dataframe with moves in underlying as columns
OTMcall_table = pd.DataFrame(index=percent_OTMs, columns=percent_moves)

# Update the dataframe with the new titles for columns and rows
OTMcall_table.columns.name = 'Moves in Underlying'
OTMcall_table.index.name = '% OTM'

for i in OTMcall_table.index:
    for j in OTMcall_table.columns:
        OTMcall_table.at[i, j] = OTMcall_T(S_0, j, i, prems[np.where(OTMcall_table.index == i)[0][0]])

In [46]:
OTMcall_table

Moves in Underlying,2.5,5.0,7.5,10.0
% OTM,,,,
3,0,6.768171,15.228384,23.688598
5,0,0,49.552679,99.105357
10,0,0,0,0
15,0,0,0,0


#### Combnined Strategy Table

In [47]:
# Calculates payoff at expiry of %OTM put given some % move in the underlying
# Note that moves in the underlying and %OTM are input as positive numbers
def OTMput_T(S_0, move_und, OTM, prem):
    E = S_0 * (1 - OTM / 100) # Strike as %OTM of S_0
    S_T = S_0 * (1 - move_und / 100) # Underlying price at expiry as % move in spot price
    #print("Strike:", E)
    #print("S_T:", S_T)
    return max(0, (E - S_T) / prem)

# Calculates payoff at expiry of %OTM call given some % move in the underlying
# Note that moves in the underlying and %OTM are input as positive numbers
def OTMcall_T(S_0, move_und, OTM, prem):
    E = S_0 * (1 + OTM / 100) # Strike as %OTM of S_0
    S_T = S_0 * (1 + move_und / 100) # Underlying price at expiry as % move in spot price
    #print("Strike:", E)
    #print("S_T:", S_T)
    return max(0, (S_T - E) / prem)

In [48]:
# Spot price
S_0 = 554.99

# %OTM put option
percent_moves = np.array([2.5, 5, 7.5, 10])
percent_OTMs = np.array([0, 3, 5, 10, 15])

# Array of premia in dollars for puts
prems_puts = np.array([6.33, 2.09, 1.09, 0.37, 0.08])

# Create empty dataframe with moves in underlying as columns
OTMput_table = pd.DataFrame(index=percent_OTMs, columns=percent_moves)

# Update the dataframe with the new titles for columns and rows
OTMput_table.columns.name = 'Moves in Underlying'
OTMput_table.index.name = '% OTM'

for i in OTMput_table.index:
    for j in OTMput_table.columns:
        OTMput_table.at[i, j] = round(OTMput_T(S_0, j, i, prems_puts[np.where(OTMput_table.index == i)[0][0]]), 2)

# %OTM call option
# Array of premia in dollars for calls
prems_calls = np.array([8.41, 1.64, 0.28, 0.08, 0.04])

# Create empty dataframe with moves in underlying as columns
OTMcall_table = pd.DataFrame(index=percent_OTMs, columns=percent_moves)

# Update the dataframe with the new titles for columns and rows
OTMcall_table.columns.name = 'Moves in Underlying'
OTMcall_table.index.name = '% OTM'

for i in OTMcall_table.index:
    for j in OTMcall_table.columns:
        OTMcall_table.at[i, j] = round(OTMcall_T(S_0, j, i, prems_calls[np.where(OTMcall_table.index == i)[0][0]]), 2)

In [54]:
percent_moves

array([ 2.5,  5. ,  7.5, 10. ])

In [49]:
OTMcall_table

Moves in Underlying,2.5,5.0,7.5,10.0
% OTM,,,,
0,1.65,3.3,4.95,6.6
3,0,6.77,15.23,23.69
5,0,0,49.55,99.11
10,0,0,0,0
15,0,0,0,0


In [50]:
OTMput_table

Moves in Underlying,2.5,5.0,7.5,10.0
% OTM,,,,
0,2.19,4.38,6.58,8.77
3,0,5.31,11.95,18.59
5,0,0,12.73,25.46
10,0,0,0,0
15,0,0,0,0


### Excel

In [51]:
import xlwings as xw

In [11]:
xw.Range('B1:B4').value

[2.0, 'Inputs', 'Underlying', 'SPY']

In [16]:
test = np.array([1,2,3]).T

In [56]:
xw.Range('A1:A3').value = [[1], [2], [3]]

In [52]:
OTMcall_table

Moves in Underlying,2.5,5.0,7.5,10.0
% OTM,,,,
0,1.65,3.3,4.95,6.6
3,0,6.77,15.23,23.69
5,0,0,49.55,99.11
10,0,0,0,0
15,0,0,0,0


In [53]:
xw.Range('B15:F20').value = OTMcall_table